In [ ]:
import utm
import pandas as pd
import itertools
from datetime import datetime
import numpy as np
import time
from scipy.optimize import linear_sum_assignment
t_start = time.time()
df1_original = pd.read_csv('/Users/mac/Desktop/st9_sample_data/st9_6.csv')
df1 = df1_original

distance_max = 6.0    # in unit of km
time_max = 5.2       # in unit of minutes

def dist(n1, n2):
    x1, y1 = utm.from_latlon(df1.iloc[n1]['dropoff_latitude'],df1.iloc[n1]['dropoff_longitude'])[0:2]
    x2, y2 = utm.from_latlon(df1.iloc[n2]['dropoff_latitude'],df1.iloc[n2]['dropoff_longitude'])[0:2]
    distance = (np.sqrt((x2-x1)**2 + (y2-y1)**2))/1000
    if distance == 0:
        dist_km = float('inf')
    elif distance > distance_max:
        dist_km = float('inf')
    else:
        dist_km = distance
    return dist_km


# In[491]:

def capa(n1, n2):
    c1 = df1.iloc[n1]['passenger_count']
    c2 = df1.iloc[n2]['passenger_count']
    if c1 + c2 > 4:
        total = float('inf')
    else:
        total = 1
    return total


# In[492]:

def timing(n1, n2):
    t1_str = str(df1.iloc[n1]['pk_datetime'])
    t2_str = str(df1.iloc[n2]['pk_datetime'])
    t1_temp = datetime.strptime(t1_str, "%Y-%m-%d %H:%M:%S")
    t2_temp = datetime.strptime(t2_str, "%Y-%m-%d %H:%M:%S")
    t1 = time.mktime(t1_temp.timetuple())
    t2 = time.mktime(t2_temp.timetuple())
    tdelta = abs(t1 - t2)
    mindelta = tdelta/60
    if mindelta > time_max:
        m = float('inf')
    else:
        m = (mindelta + 1)/(time_max + 1)
    return m


# In[493]:

def cost(n1, n2):
    cost = dist(n1, n2) * capa(n1, n2) * timing(n1, n2)
    return cost

#********


'''
--------------
'''

# In[494]:
t_1 = time.time()

matrix_3 = np.zeros((len(df1.index), len(df1.index)))       # replace with matrix 1 in old code
max_ind = len(df1.index)
index_array = list(sorted(itertools.combinations(range(max_ind), 2)))
for id, value in enumerate(index_array):
     row, col = value
     matrix_3[row][col] = cost(row, col)
matrix_3[np.where(matrix_3 == float('inf'))] = 10000.0
out2 = matrix_3 + matrix_3.T + np.eye(max_ind)*(10000.0 - 1)
t_2 = time.time()
print(' read by new code ' + str(t_2 - t_1))
# ------------------- convert inf to large value

matrix_2 = out2

row_ind, col_ind = linear_sum_assignment(matrix_2)
from scipy.optimize import linear_sum_assignment
row_ind, col_ind = linear_sum_assignment(matrix_2)

t_final = time.time()
print('optimal allocation is ' + str(col_ind))
print('total cost for matching is ' + str(matrix_2[row_ind, col_ind].sum()))
print("total time spent is: "+str(t_final - t_start))
i = 0
number = 0
while i < len(col_ind):
    if col_ind[i]!= i:
        number = number + 1
    i = i + 1
print('total matching number is:' + str(number))

with open('/Users/mac/Desktop/monthly_matching/matching6.csv', 'wb') as f:
    f.write(b'header\n')
    np.savetxt(f, col_ind, delimiter=",")
